In [1]:
import logging
import sys
import os
import pandas as pd
from pandas import Timestamp

# pylint: disable=import-error disable=wrong-import-position
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))

print(os.path.dirname(os.path.abspath("__file__")))
import pydit

logger = pydit.start_logging_info()
logger.info("Started")
logger.error("Some error")
logger.debug("Some debug")
logger.warning("you be warned")



/workspaces/pydit/examples
2023-08-28 14:57:41 -  INFO      - Logging started at INFO level
2023-08-28 14:57:41 -  INFO      - Started
2023-08-28 14:57:41 -  ERROR     - Some error
2023-08-28 14:57:41 -  WARNING   - you be warned
